<a href="https://colab.research.google.com/github/JsuccessJ/dialect_transformer/blob/main/basic_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
#설치 후 런타임 재실행해야함
!pip install accelerate>=0.20.1


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 29.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=1beae19b6821a60d776c4fe4e81f4b24bc82467ca95ca6a0d8a3c93e535518c6
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import wandb
!wandb login "f2d7cd4fedd2822e534395005a236ac7b01e0036"
wandb.init(project="dialect_transformer")

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: hjs3545 (nlp_learning). Use `wandb login --relogin` to force relogin


In [ ]:
wandb.run.name = '경상도3만'
wandb.run.save()

wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [ ]:
# Hyperparameters
batch_size = 32
learning_rate = 0.001
epochs = 10

args = {
    "learning_rate": learning_rate,
    "epochs": epochs,
    "batch_size": batch_size
}
wandb.config.update(args)

In [ ]:
#라이브러리 총집합
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import PreTrainedTokenizerFast, BartModel,BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')

model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2')

device = torch.device("cuda:0")


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
model = model.to(device)

In [ ]:
!nvidia-smi

Sat Oct 14 14:35:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    52W / 400W |   1587MiB / 40960MiB |     21%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/hjse/jaesung/dataset_경상도3만_train.csv')

In [ ]:
data.head()


,standard,dialect
0,머 못 먹 그리고 모든걸 다 좋아하는 데 즐기진 않는다.,머 못 묵 그라고 모든걸 다 좋아하는 데 즐기진 않는다.
1,근데 먹이가 있어야지 얘네가 먹고 자랄꺼 아니야.,근데 먹이가 있어야지 얘네가 먹고 자랄꺼 아이가.
2,먹이가 될수 없데 근데 머가 얘들의 먹이는 무엇이냐면 식이섬유,먹이가 될수 없데 근데 머가 얘들의 먹이는 머냐면 식이섬유
3,그러니깐 인제 야채 과일 머 식이섬유 머 자기는 그런거 좋아하니깐,그러니깐 인자 야채 과일 머 식이섬유 머 자기는 그런거 좋아하니깐
4,아니 어쩔수 없이 먹는거 빼고 음식점 선 음식점 선택하기도 그렇고 같이 그 머 식사...,아니 어쩔수 없이 먹는거 빼고 음식점 선 음식점 선택하기도 그렇고 같이 그 머 식사...


In [ ]:
# 1열(첫 번째 열)에서 가장 긴 문자열의 길이 찾기
max_length = data.iloc[:, 0].str.len().max()

# 가장 긴 문자열 찾기
longest_sentence = data.loc[data.iloc[:, 0].str.len() == max_length, data.columns[0]].values[0]

print("가장 긴 문장의 글자수:", max_length)
print("가장 긴 문장을 출력함:", longest_sentence)

가장 긴 문장의 글자수: 151
가장 긴 문장을 출력함: 그래 가지구 사진으로 시선을 끌고 여기에서 더 깊은 이야기를 할 수 있으면 좋은데 집중력이 너무 짧아졌으니 유투브 시대도 지나고 틱톡이 지금 다행히 이게 다행이라고 해야 될지는 모르겠는데 틱톡도 지금 뭔가 조금 나라 간의 이슈 그런것 좀 되게 문제가 많은 플랫폼이잖아요.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1열(standard)과 2열(dialect)을 각각 features(X)와 labels(y)로 설정
x = data['standard']
y = data['dialect']

# train 데이터와 test 데이터로 나누기 (4:1 비율)
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)

# 나눈 데이터의 크기 확인 (예시로 출력)
print("Train 데이터 크기:", x_train.shape[0])
print("Valid 데이터 크기:", x_valid.shape[0])


Train 데이터 크기: 24000
Valid 데이터 크기: 6000


In [ ]:
import torch
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

# dialectDataset 클래스 정의
class dialectDataset(Dataset):
    def __init__(self, standard, dialect, tokenizer):
        self.standard = standard
        self.dialect = dialect
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.standard)  # 또는 len(self.dialect) 사용해도 됨


    def __getitem__(self, idx):
        standard_text = self.standard[idx]
        dialect_text = self.dialect[idx]

        # 토크나이징 및 패딩 수행
        inputs = self.tokenizer(
            text=dialect_text,
            padding="max_length",  # 필요한 경우 max_length 설정
            truncation=True,       # 필요한 경우 문장 자르기 설정
            return_tensors="pt",   # PyTorch 텐서로 반환
            max_length=160,
            )

        outputs = self.tokenizer(
            text=standard_text,
            padding="max_length",  # 필요한 경우 max_length 설정
            truncation=True,       # 필요한 경우 문장 자르기 설정
            return_tensors="pt",   # PyTorch 텐서로 반환
            max_length=130,
            )

        # 모델 입력을 포함하는 딕셔너리 반환
        sample = {
            'input_ids': inputs.input_ids.squeeze(),
            'attention_mask': inputs.attention_mask.squeeze(),
            'labels': outputs.input_ids.squeeze()  # 예시로 labels에 input_ids를 사용하도록 설정
        }

        return sample



# dialectDataset 클래스를 사용하여 훈련 및 검증 데이터셋 생성
train_dataset = dialectDataset(x_train.values, y_train.values,tokenizer)
valid_dataset = dialectDataset(x_valid.values, y_valid.values,tokenizer)









In [ ]:
# 첫 번째 샘플 얻기
first_sample = train_dataset[0]
print(first_sample)


{'input_ids': tensor([21675, 17582, 15614, 20110, 14364,  9229, 14339, 19303, 15709, 16513,
        25282, 20047, 14390,  9049,   245,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3, 

In [ ]:
# first_standard_text = train_dataset.standard[23999]
# first_dialect_text = train_dataset.dialect[23999]

# print("Standard Text:", first_standard_text)
# print("Dialect Text:", first_dialect_text)


In [ ]:
from transformers import TrainingArguments
from transformers import Trainer
from transformers import DataCollatorWithPadding
from transformers import EvalPrediction

# 평가함수 정의 (bleu score 출력)
def compute_metrics(p: EvalPrediction):
    preds = p.predictions
    labels = p.label_ids

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # BLEU 스코어 계산을 위해 두 개의 리스트로 만들기
    references = [[label.strip()] for label in decoded_labels]
    predictions = [pred.strip() for pred in decoded_preds]

    # BLEU 스코어 계산
    from nltk.translate.bleu_score import corpus_bleu
    bleu_score = corpus_bleu(references, predictions)

    return {"bleu_score": bleu_score}


# 파인튜닝을 위한 훈련 설정
training_args = TrainingArguments(
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    output_dir="/content/drive/MyDrive/hjse/jaesung/pretrained_경상도3만",
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/hjse/jaesung/pretrained_경상도3만",
    logging_steps=10,
    num_train_epochs=10,
    save_total_limit=1,  # 저장할 최대 모델 수, 1개만 자동으로 저장
    report_to='wandb'
)

# automatically log model to W&B at the end
%env WANDB_LOG_MODEL=true

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics

)


env: WANDB_LOG_MODEL=true


In [ ]:
# 파인튜닝 실행
trainer.train()

# 파인튜닝된 모델 저장
trainer.save_model()

# wandb 종료
wandb.finish()


Step,Training Loss
10,17.495800
20,15.367900
30,11.652900
40,8.831200
50,6.888300
60,5.442900
70,4.721000
80,4.233600
90,3.843000
100,3.452200


train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,▂▄▇███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,10.0
train/global_step,7500
